### This notebook run from WSL, 

#### Enviroment 
* experiment1-env - <code>source /home/kali/experiment1-env/bin/activate</code>

* workingspace <code>/mnt/e/Onedrive/1-Documents/1-PhD/2-Experiment/feature_extract</code>

#### Dataset 
* UNSW - <code>/mnt/f/My Drive/UNSW_pcap-isolated</code>
* MonIoTr - <code>/mnt/d/Dataset/iot-data/us</code>
* Sentinel - <code>/mnt/e/Onedrive/1-Documents/1-PhD/2-Experiment/IoT_Sentinel-master/captures_IoT-Sentinel</code>

#### CSV
* nfstream extract - <code>/mnt/f/My Drive/IoT_Dataset</code>

In [1]:
import nfstream
print(nfstream.__version__)

6.3.5


In [1]:
import pandas as pd
import glob, os
import numpy as np
from nfstream import NFStreamer, NFPlugin
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [8]:
source="/mnt/f/My Drive/UNSW_pcap-isolated/Amazon Echo/1.pcap"

In [9]:
df = NFStreamer(source,statistical_analysis=True).to_pandas()
df.head()

,id,expiration_id,src_ip,src_mac,src_oui,src_port,dst_ip,dst_mac,dst_oui,dst_port,protocol,ip_version,vlan_id,tunnel_id,bidirectional_first_seen_ms,bidirectional_last_seen_ms,bidirectional_duration_ms,bidirectional_packets,bidirectional_bytes,src2dst_first_seen_ms,src2dst_last_seen_ms,src2dst_duration_ms,src2dst_packets,src2dst_bytes,dst2src_first_seen_ms,dst2src_last_seen_ms,dst2src_duration_ms,dst2src_packets,dst2src_bytes,bidirectional_min_ps,bidirectional_mean_ps,bidirectional_stddev_ps,bidirectional_max_ps,src2dst_min_ps,src2dst_mean_ps,src2dst_stddev_ps,src2dst_max_ps,dst2src_min_ps,dst2src_mean_ps,dst2src_stddev_ps,dst2src_max_ps,bidirectional_min_piat_ms,bidirectional_mean_piat_ms,bidirectional_stddev_piat_ms,bidirectional_max_piat_ms,src2dst_min_piat_ms,src2dst_mean_piat_ms,src2dst_stddev_piat_ms,src2dst_max_piat_ms,dst2src_min_piat_ms,dst2src_mean_piat_ms,dst2src_stddev_piat_ms,dst2src_max_piat_ms,bidirectional_syn_packets,bidirectional_cwr_packets,bidirectional_ece_packets,bidirectional_urg_packets,bidirectional_ack_packets,bidirectional_psh_packets,bidirectional_rst_packets,bidirectional_fin_packets,src2dst_syn_packets,src2dst_cwr_packets,src2dst_ece_packets,src2dst_urg_packets,src2dst_ack_packets,src2dst_psh_packets,src2dst_rst_packets,src2dst_fin_packets,dst2src_syn_packets,dst2src_cwr_packets,dst2src_ece_packets,dst2src_urg_packets,dst2src_ack_packets,dst2src_psh_packets,dst2src_rst_packets,dst2src_fin_packets,application_name,application_category_name,application_is_guessed,requested_server_name,client_fingerprint,server_fingerprint,user_agent,content_type
0,0,0,192.168.1.240,44:65:0d:56:cc:d3,44:65:0d,37543,207.171.178.6,14:cc:20:51:33:ea,14:cc:20,123,17,4,0,0,1474552870492,1474552924023,53531,4,360,1474552870492,1474552923879,53387,2,180,1474552870636,1474552924023,53387,2,180,90,90.000000,0.000000,90,90,90.0,0.0,90,90,90.0,0.000000,90,144,17843.666667,30656.721944,53243,53387,53387.000000,0.000000,53387,53387,53387.0,0.000000,53387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NTP.Amazon,System,0,NaN,NaN,NaN,NaN,NaN
1,1,0,192.168.1.240,44:65:0d:56:cc:d3,44:65:0d,0,208.67.220.220,14:cc:20:51:33:ea,14:cc:20,0,1,4,0,0,1474552945680,1474552945701,21,10,980,1474552945680,1474552945700,20,5,490,1474552945681,1474552945701,20,5,490,98,98.000000,0.000000,98,98,98.0,0.0,98,98,98.0,0.000000,98,1,2.333333,2.061553,7,3,5.000000,2.160247,8,3,5.0,2.160247,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ICMP.OpenDNS,Network,0,NaN,NaN,NaN,NaN,NaN
2,2,0,192.168.1.240,44:65:0d:56:cc:d3,44:65:0d,34378,8.8.8.8,14:cc:20:51:33:ea,14:cc:20,53,17,4,0,0,1474552945882,1474552945884,2,4,344,1474552945882,1474552945882,0,2,150,1474552945884,1474552945884,0,2,194,75,86.000000,13.613719,103,75,75.0,0.0,75,91,97.0,8.485281,103,0,0.666667,1.154701,2,0,0.000000,0.000000,0,0,0.0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DNS.Google,Web,0,www.example.net,NaN,NaN,NaN,NaN
3,3,0,192.168.1.240,44:65:0d:56:cc:d3,44:65:0d,57787,8.8.8.8,14:cc:20:51:33:ea,14:cc:20,53,17,4,0,0,1474552946070,1474552946072,2,4,344,1474552946070,1474552946070,0,2,150,1474552946072,1474552946072,0,2,194,75,86.000000,13.613719,103,75,75.0,0.0,75,91,97.0,8.485281,103,0,0.666667,1.154701,2,0,0.000000,0.000000,0,0,0.0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DNS.Google,Web,0,www.example.org,NaN,NaN,NaN,NaN
4,4,0,192.168.1.240,44:65:0d:56:cc:d3,44:65:0d,41356,93.184.216.34,14:cc:20:51:33:ea,14:cc:20,80,6,4,0,0,1474552946085,1474552946383,298,6,412,1474552946085,1474552946383,298,4,272,1474552946229,1474552946375,146,2,140,66,68.666667,4.131182,74,66,68.0,4.0,74,66,70.0,5.656854,74,1,59.600000,76.637458,144,1,99.333333,85.195853,151,146,146.0,0.000000,146,2,0,0,0,5,0,0,2,1,0,0,0,3,0,0,1,1,0,0,0,2,0,0,1,HTTP,Web,1,NaN,NaN,NaN,NaN,NaN


## 1-Load Dataset

In [11]:
dataset = pd.read_csv("non_iot_dataset.csv")
dataset.shape

/home/kali/experiment1-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(472983, 39)

In [15]:
dataset.columns

Index(['id', 'expiration_id', 'src_ip', 'src_mac', 'src_oui', 'src_port',
       'dst_ip', 'dst_mac', 'dst_oui', 'dst_port', 'protocol', 'ip_version',
       'vlan_id', 'tunnel_id', 'bidirectional_first_seen_ms',
       'bidirectional_last_seen_ms', 'bidirectional_duration_ms',
       'bidirectional_packets', 'bidirectional_bytes', 'src2dst_first_seen_ms',
       'src2dst_last_seen_ms', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_first_seen_ms', 'dst2src_last_seen_ms',
       'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'application_name', 'application_category_name',
       'application_is_guessed', 'requested_server_name', 'client_fingerprint',
       'server_fingerprint', 'user_agent', 'content_type', 'device', 'label'],
      dtype='object')

In [79]:
dataset['device'].unique()

array(['Withings Smart Baby Monitor', 'Blipcare Blood Pressure meter',
       'Withings Aura smart sleep sensor', 'PIX-STAR Photo-frame',
       'Netatmo weather station', 'Smart Things', 'Dropcam',
       'Amazon Echo', 'TP-Link Smart plug', 'Insteon Camera',
       'NEST Protect smoke alarm', 'HP Printer', 'IoT Camera1',
       'Light Bulbs LiFX Smart Bulb', 'Withings Smart scale',
       'Samsung SmartCam', 'Belkin wemo motion sensor', 'Netatmo Welcome',
       'Belkin Wemo switch', 'Triby Speaker',
       'TP-Link Day Night Cloud camera', 'iHome Power Plug',
       'Android Phone 1', 'Android Phone 2', 'IPhone', 'Laptop',
       'MacBook', 'Samsung Galaxy Tab', 'MacBook Iphone'], dtype=object)

## 2-Feature Selection

## 3-Model Training

In [81]:

from sklearn.ensemble import RandomForestClassifier

df = dataset
X = df[["bidirectional_packets", "bidirectional_bytes"]]
y = df["label"]#.apply(lambda x: 1 if 'System' in x else 0)
model = RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier()

## 4-Model Prediction

In [103]:
from nfstream import NFPlugin, NFStreamer
import numpy
source= "/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 1/47.pcap"
class ModelPrediction(NFPlugin):
    def on_init(self, packet, flow):
        flow.udps.model_prediction = 0
      
    def on_expire(self, flow):
         
        to_predict = numpy.array([flow.bidirectional_packets,
                                  flow.bidirectional_bytes]).reshape((1,-1))
       
        flow.udps.model_prediction = self.my_model.predict(to_predict)
        
ml_streamer = NFStreamer(source, udps=ModelPrediction(my_model=model))
Y=lambda x: 'iot' if 1 in x else 'non-iot'
#test_X=[]
test_y=[]
y_pred=[]
for flow in ml_streamer:
    y=Y(flow.udps.model_prediction)
    #test_X.append(flow.to_predict)
    test_y.append('non-iot')
    y_pred.append(y)
    print(flow.udps.model_prediction,y)

[1] iot
[0] non-iot
[1] iot
[1] iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[1] iot
[0] non-iot
[1] iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[1] iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[1] iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[1] iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[1] iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-iot
[0] non-

In [104]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(test_y,y_pred))
print(classification_report(test_y,y_pred))

[[ 0  0]
 [34 97]]
              precision    recall  f1-score   support

         iot       0.00      0.00      0.00         0
     non-iot       1.00      0.74      0.85       131

    accuracy                           0.74       131
   macro avg       0.50      0.37      0.43       131
weighted avg       1.00      0.74      0.85       131



/home/kali/experiment1-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [119]:
%env

{'SHELL': '/bin/bash',
 'WSL_DISTRO_NAME': 'kali-linux',
 'WT_SESSION': 'b47c5d7d-4fe6-453a-b729-c631cadba628',
 'LESS_TERMCAP_se': '\x1b',
 'LESS_TERMCAP_so': '\x1b',
 'NAME': 'DESKTOP-NVF7L0S',
 'PWD': '/mnt/e/OneDrive/1-Documents/1-PhD/2-Experiment/feature_extract',
 'LOGNAME': 'kali',
 'HOME': '/home/kali',
 'LANG': 'en_US.UTF-8',
 'WSL_INTEROP': '/run/WSL/11_interop',
 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;